In [1]:
import psycopg2

In [2]:
conn_string = "host='HOST' dbname='startml' user='robot-startml-ro' password='' port='6432'"

In [10]:
def get_query_result(conn_string,
                     query,
                     conn=None):
    
    if conn is None:
        with psycopg2.connect(conn_string) as conn:
            cursor = conn.cursor()
            cursor.execute(query)
            res = cursor.fetchall()
    else:
        cursor = conn.cursor()
        cursor.execute(query)
        res = cursor.fetchall()
        
    return res

In [26]:
def print_result(query):
    global conn_string
    print(get_query_result(conn_string, query, None))

In [23]:
query = \
    """
    SELECT * 
      FROM "user" 
     LIMIT 1
    """

In [27]:
print_result(query)

[(200, 1, 34, 'Russia', 'Degtyarsk', 3, 'Android', 'ads')]


In [29]:
query = \
    """
    SELECT COUNT(DISTINCT topic), 
           MAX(LENGTH(topic)) 
      FROM post
    """

In [30]:
print_result(query)

[(7, 13)]


In [33]:
query = \
    """
    SELECT distinct topic
      FROM post
     WHERE LENGTH(topic) = 13
    """

In [34]:
print_result(query)

[('entertainment',)]


In [40]:
query = \
    """
    SELECT *
      FROM "user"
     WHERE age > 30
       AND os = 'iOS'
     LIMIT 1
    """

In [41]:
print_result(query)

[(212, 1, 47, 'Russia', 'Podolsk', 0, 'iOS', 'ads')]


In [47]:
query = \
    """
    SELECT id
      FROM "user"
     WHERE country != 'Russia'
       AND (exp_group not in (0, 3) or city = 'Minsk')
     LIMIT 4
    """

In [48]:
print_result(query)

[(211,), (213,), (216,), (218,)]


In [51]:
query = \
    """
    SELECT country, ROUND(AVG(age), 2)
      FROM "user"
     GROUP BY country
    """

In [52]:
print_result(query)

[('Azerbaijan', Decimal('26.78')), ('Belarus', Decimal('27.28')), ('Cyprus', Decimal('28.07')), ('Estonia', Decimal('28.15')), ('Finland', Decimal('27.13')), ('Kazakhstan', Decimal('27.22')), ('Latvia', Decimal('27.22')), ('Russia', Decimal('27.19')), ('Switzerland', Decimal('27.95')), ('Turkey', Decimal('27.45')), ('Ukraine', Decimal('27.26'))]


In [55]:
query = \
    """
    SELECT exp_group, os,
           COUNT(id) total_users,
           MAX(age) max_age,
           MIN(age) min_age
     GROUP BY exp_group, os
    """

In [56]:
print_result(query)

[(0, 'Android', 21234, 95, 14), (0, 'iOS', 11489, 79, 14), (1, 'Android', 21232, 92, 14), (1, 'iOS', 11406, 87, 14), (2, 'Android', 21102, 78, 14), (2, 'iOS', 11512, 74, 14), (3, 'Android', 21319, 84, 14), (3, 'iOS', 11449, 85, 14), (4, 'Android', 21085, 84, 14), (4, 'iOS', 11377, 78, 14)]


In [57]:
query = \
    """
    SELECT topic, MAX(LENGTH(text))
      FROM post
     GROUP BY topic
    """

In [58]:
print_result(query)

[('tech', 16063), ('covid', 148), ('movie', 10234), ('entertainment', 19004), ('politics', 25392), ('sport', 9438), ('business', 5378)]


In [59]:
query = \
    """
    SELECT country, COUNT(id)
      FROM "user"
     GROUP BY country
    HAVING COUNT(id) > 1000
     ORDER BY COUNT(id) ASC
    """

In [60]:
print_result(query)

[('Azerbaijan', 1542), ('Finland', 1599), ('Turkey', 1606), ('Kazakhstan', 3172), ('Belarus', 3293), ('Ukraine', 8273), ('Russia', 143035)]


In [64]:
query = \
    """
    SELECT COUNT(id)
      FROM "user"
     WHERE city = 'Moscow'
     GROUP BY exp_group
    HAVING AVG(age) > 27.2
    """

In [65]:
print_result(query)

[(4414,), (4378,)]


In [70]:
query = \
    """
    SELECT topic
      FROM post
     GROUP BY topic
     ORDER BY COUNT(id) DESC
     LIMIT 3
    """

In [71]:
print_result(query)

[('movie',), ('covid',), ('sport',)]


In [74]:
query = \
    """
    SELECT id
      FROM "user"
     WHERE city = 'Voronezh'
     ORDER BY age DESC, exp_group ASC
     LIMIT 2
    """

In [75]:
print_result(query)

[(89990,), (2441,)]


In [76]:
query = \
    """
    SELECT fa.post_id, fa.time, 
           u.age, u.os
      FROM feed_action fa 
           JOIN "user" u on fa.user_id = u.id
     WHERE u.city = 'Omsk'
       AND action = 'like'
     ORDER BY fa.time DESC
     LIMIT 100
    """

In [ ]:
print_result(query)

In [77]:
query = \
    """
    SELECT city
      FROM feed_action fa 
           JOIN "user" u ON fa.user_id = u.id
           JOIN post p ON p.id = fa.post_id
     WHERE fa.time::DATE = '2021-12-01'
       AND u.age = 36
       AND p.topic = 'covid'
       AND fa.action = 'view'
     GROUP BY city 
     ORDER BY COUNT(fa.action) ASC
     LIMIT 1
    """

In [78]:
print_result(query)

[('Arkhangelsk',)]


In [83]:
query = \
    """
    SELECT post_id, 
           COUNT(DISTINCT user_id),
           MAX(CASE WHEN action = 'like' THEN time ELSE NULL END)
      FROM feed_action
     GROUP BY post_id
     ORDER BY SUM(CASE WHEN action = 'like' THEN 1 ELSE 0 END) DESC
     LIMIT 3
    """

In [ ]:
print_result(query)